In [1]:
from dotenv import load_dotenv
import asyncio

load_dotenv()

True

In [ ]:
import os

In [ ]:
import requests
import json


def stream_with_reasoning(question, api_token):
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_token}",
        "Content-Type": "application/json",
    }

    payload = {
        "model": "deepseek/deepseek-r1",
        "messages": [{"role": "user", "content": question}],
        "stream": True,
        "include_reasoning": True,
    }

    buffer = ""
    is_reasoning = True  # Track whether we're in reasoning or content phase
    print("Reasoning:", end="\n\n", flush=True)

    with requests.post(url, headers=headers, json=payload, stream=True) as r:
        for chunk in r.iter_content(chunk_size=1024, decode_unicode=True):
            buffer += chunk
            while True:
                try:
                    line_end = buffer.find("\n")
                    if line_end == -1:
                        break

                    line = buffer[:line_end].strip()
                    buffer = buffer[line_end + 1 :]

                    if line.startswith("data: "):
                        data = line[6:]
                        if data == "[DONE]":
                            break

                        try:
                            data_obj = json.loads(data)
                            delta = data_obj["choices"][0].get("delta", {})

                            # Check for phase transition
                            if (
                                is_reasoning
                                and "content" in delta
                                and delta.get("content")
                            ):
                                is_reasoning = False
                                print("\n\nResponse:", end="\n\n", flush=True)

                            # Handle reasoning phase
                            if is_reasoning and "reasoning" in delta:
                                reasoning = delta["reasoning"]
                                if reasoning and reasoning.strip():
                                    print(reasoning, end="", flush=True)

                            # Handle content phase
                            elif not is_reasoning and "content" in delta:
                                content = delta["content"]
                                if content and content.strip():
                                    print(content, end="", flush=True)

                        except json.JSONDecodeError:
                            pass
                except Exception as e:
                    print(f"\nError: {str(e)}")
                    break


question = "How would you build the tallest building ever?"
api_token = os.getenv("OPENROUTER_API_KEY")
stream_with_reasoning(question, api_token)

Reasoning:





Response:

Building the tallest building ever is a monumental, incredibly complex undertaking, far beyond simply scaling up current skyscrapers. It would require pushing the absolute limits of engineering, materials science, construction techniques, and human ingenuity. Here's a breakdown of how you might approach such a project:

**1. Vision and Justification:**

* **Why build this tall?**  Before even thinking about engineering, you need a compelling reason. Simply being "tallest" isn't enough for the immense investment and effort.  The purpose could be:
    * **Vertical City Concept:**  Creating a self-contained environment with residential, commercial, agricultural, and recreational spaces, potentially addressing land scarcity or future environmental challenges.
    * **Technological Showcase:**  Demonstrating cutting-edge technologies and pushing the boundaries of human achievement.
    * **Research and Development Hub:**  Creating a unique research facility with extreme environ